In [ ]:
"""
Datasets are available at https://archive.ics.uci.edu/datasets 
"""

In [1]:
import sys
print(sys.version)

3.10.11 (v3.10.11:7d4cc5aa85, Apr  4 2023, 19:05:19) [Clang 13.0.0 (clang-1300.0.29.30)]


In [2]:
import pandas as pd
from silhouette_upper_bound import upper_bound, upper_bound_macro_silhouette
import numpy as np
from sklearn.metrics import adjusted_rand_score, silhouette_samples, adjusted_mutual_info_score
from collections import Counter
import kmedoids
from sklearn.preprocessing import StandardScaler, RobustScaler, normalize
from sklearn.impute import SimpleImputer
from scipy.spatial.distance import squareform, pdist
from tqdm import tqdm
from pathlib import Path
from scipy.io import arff
from scipy import stats

In [3]:
def macro_averaged_silhouette(dissimilarity_matrix, labels):
    """
    Return macro-averaged silhouette score.
    """

    silhouette_scores = silhouette_samples(
        X=dissimilarity_matrix, labels=labels, metric="precomputed"
    )

    mac_silh = []

    for cluster_id in np.unique(labels):
        scores = silhouette_scores[labels == cluster_id]

        mac_silh.append(np.mean(scores))

    return np.mean(mac_silh)

In [4]:
def save_results(path, diss_matrix, k_range = range(2, 16), random_state = 42):
    """
    Save experimental results.

    Given the input dissimilarity matrix, compare kmedoids clustering with upper bounds. 
    """

    if Path(path).exists():
        print("Path exists. Aborting.")
        return None 

    results = {}

    list_n_clusters = []
    list_cluster_labels = []
    list_cluster_sizes = []
    list_min_cluster_size = []
    list_silh_samples = []
    list_asw = []
    list_ub_asw = []
    list_ub_asw_min_cluster_size = []
    list_macro_silhouette = []
    list_ub_macro = []

    pbar = tqdm(k_range)

    for k in pbar:
        pbar.set_description(f"Number of cluster (K): {k}")

        # cluster labels
        cluster_labels = (kmedoids.pamsil(diss=diss_matrix, medoids=k, random_state=random_state).labels + 1)

        # cluster sizes
        cluster_sizes = list(Counter(cluster_labels).values())
        min_cluster_size = min(cluster_sizes)

        # silhouette samples 
        silh_samples = silhouette_samples(X=diss_matrix, labels=cluster_labels, metric='precomputed')

        # ASW 
        asw = np.mean(silh_samples)

        # upper bounds 
        ub_asw = upper_bound(diss_matrix)
        ub_asw_min_cluster_size = upper_bound(diss_matrix, m=min_cluster_size)

        # Macro silhouette 
        macro_silhouette = macro_averaged_silhouette(diss_matrix, cluster_labels)

        # Macro-silhouette upper bound
        ub_macro = upper_bound_macro_silhouette(diss_matrix, cluster_sizes)

        # Save results 
        list_n_clusters.append(k)
        list_cluster_labels.append(cluster_labels)
        list_cluster_sizes.append(cluster_sizes)
        list_min_cluster_size.append(min_cluster_size)
        list_silh_samples.append(silh_samples)
        list_asw.append(asw)
        list_ub_asw.append(ub_asw)
        list_ub_asw_min_cluster_size.append(ub_asw_min_cluster_size)
        list_macro_silhouette.append(macro_silhouette)
        list_ub_macro.append(ub_macro)
    
    results = {
        "n_clusters": list_n_clusters,
        "cluster_labels": list_cluster_labels,
        "cluster_sizes": list_cluster_sizes,
        "min_cluster_size": list_min_cluster_size,
        "silh_samples": list_silh_samples,
        "asw": list_asw,
        "ub_asw": list_ub_asw,
        "ub_asw_min_cluster_size": list_ub_asw_min_cluster_size,
        "macro_silhouette": list_macro_silhouette,
        "ub_macro": list_ub_macro
    }
    
    # Save to pickle 
    pd.DataFrame.from_dict(results).to_pickle(path)


In [5]:
def results_stats(results, n_classes, labels = None):
    """
    Summarize results.
    """

    # use GT number of clusterss
    n_classes_results = results[results["n_clusters"] == n_classes]

    min_cluster_size = n_classes_results["min_cluster_size"].values[0]

    asw = n_classes_results["asw"].values[0]
    ub_asw = n_classes_results["ub_asw"].values[0]
    wcre = (ub_asw - asw) / ub_asw

    ari, ami = None, None 
    if labels is not None:
        ari = adjusted_rand_score(labels, n_classes_results["cluster_labels"].values[0])
        ami = adjusted_mutual_info_score(labels, n_classes_results["cluster_labels"].values[0])

    ub_asw_adjusted = n_classes_results["ub_asw_min_cluster_size"].values[0]
    wcre_adjusted = (ub_asw_adjusted - asw) / ub_asw_adjusted

    measurements = [n_classes_results["n_clusters"].values[0], min_cluster_size, ari, ami, asw, ub_asw, wcre, ub_asw_adjusted, wcre_adjusted]

    headers = ["K", "min|C|", "ARI", "AMI", "ASW", "UB", "WCRE", "UBm", "WCREm"]

    w = 10
    print("".join(f"{h:<{w}}" for h in headers))
    print("-" * (w * len(headers)))
    print("".join(f"{m:<{w}.3f}" if isinstance(m, (int, float)) else f"{str(m):<{w}}" for m in measurements))

In [6]:
base_path = Path.cwd()

In [7]:
# generate figures folder
figures_dir = base_path / "figures"
figures_dir.mkdir(parents=True, exist_ok=True)

# generate results folder
results_dir = base_path / "results"
results_dir.mkdir(parents=True, exist_ok=True)

# generate arrays folder
arrays_dir = base_path / "arrays"
arrays_dir.mkdir(parents=True, exist_ok=True)

# Ceramic shards 
* Scaling: Standard scaler
* Outlier removal: None
* n_samples: 88
* n_features: 17
* n_classes: 2
* Access to labels: yes
* Metric: L1 distance

In [49]:
# Load dataset
df = pd.read_csv(f"{base_path}/datasets/ceramic/data.csv")

In [50]:
# Select only numeric chemical composition columns
y = df["Part"]
X = df.drop(columns=["Ceramic Name", "Part"])
X_scaled = StandardScaler().fit_transform(X)
X_scaled.shape 

(88, 17)

In [51]:
np.save(f"{base_path}/arrays/ceramic.npy", X_scaled)
D = squareform(pdist(X_scaled, metric="cityblock"))

In [52]:
# save results 
save_results(path = f"{base_path}/results/ceramic.pkl", diss_matrix=D)

Path exists. Aborting.


In [53]:
results_stats(results=pd.read_pickle(f"{base_path}/results/ceramic.pkl") , n_classes=2, labels=y)

K         min|C|    ARI       AMI       ASW       UB        WCRE      UBm       WCREm     
------------------------------------------------------------------------------------------
2         44        1.000     1.000     0.358     0.621     0.423     0.403     0.112     


# Gene expressions
* Scaling: None
* Outlier removal: None
* n_samples: 801
* n_features: 20531
* n_classes: 5
* Access to labels: no
* Metric: L2 distance

In [54]:
df = pd.read_csv(f"{base_path}/datasets/rna/data.csv")

In [55]:
data = df.select_dtypes(include="number")
X = data.to_numpy()
X.shape

(801, 20531)

In [56]:
np.save(f"{base_path}/arrays/rna.npy", X)
D = squareform(pdist(X, metric="euclidean"))

In [57]:
save_results(path = f"{base_path}/results/rna.pkl", diss_matrix=D)

Path exists. Aborting.


In [58]:
results_stats(results=pd.read_pickle(f"{base_path}/results/rna.pkl"), n_classes = 5)

K         min|C|    ARI       AMI       ASW       UB        WCRE      UBm       WCREm     
------------------------------------------------------------------------------------------
5         77        None      None      0.225     0.419     0.464     0.323     0.303     


# Wine 
* Scaling: Standard Scaler
* Outlier removal: None
* n_samples: 178
* n_features: 13
* n_classes: 3
* Access to labels: yes
* Metric: L1 distance

In [59]:
data, meta = arff.loadarff(f"{base_path}/datasets/wine/wine.arff")
df = pd.DataFrame(data)

In [60]:
# Frst column is label
y = df.iloc[:, 0].astype(str).to_numpy()
X = df.iloc[:, 1:].to_numpy()

In [61]:
X = StandardScaler().fit_transform(X)
X.shape 

(178, 13)

In [62]:
np.save(f"{base_path}/arrays/wine.npy", X)
D = squareform(pdist(X, metric="cityblock"))

In [63]:
save_results(path = f"{base_path}/results/wine.pkl", diss_matrix=D)

Number of cluster (K): 15: 100%|██████████| 14/14 [00:20<00:00,  1.46s/it]


In [64]:
results_stats(results = pd.read_pickle(f"{base_path}/results/wine.pkl"), n_classes = 3, labels = y)

K         min|C|    ARI       AMI       ASW       UB        WCRE      UBm       WCREm     
------------------------------------------------------------------------------------------
3         53        0.884     0.864     0.315     0.649     0.514     0.450     0.298     


# WDBC
* Scaling: Standard Scaler
* Outlier removal: Yes
* n_samples: 382
* n_features: 30
* n_classes: 2
* Access to labels: yes
* Metric: L2 distance

In [66]:
# --- 1. Load the ARFF file ---
data, meta = arff.loadarff(f"{base_path}/datasets/wdbc/wdbc.arff")

# --- 2. Convert to DataFrame ---
df = pd.DataFrame(data)

# --- 3. Decode bytes (ARFF nominal fields are byte strings) ---
df = df.applymap(lambda x: x.decode("utf-8") if isinstance(x, bytes) else x)

# --- 4. Drop ID column (not a feature) ---
df = df.drop(columns=["IDNumber"])

# --- 5. Split features and class label ---
X = df.drop(columns=["class"]).astype(float)
y = df["class"]

# --- 6. Standardize features for Euclidean distance ---
# StandardScaler makes all features comparable (zero mean, unit variance)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

/var/folders/40/l50vx_fn2nd9hv4y_ssvkhn80000gn/T/ipykernel_59080/3879370263.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.decode("utf-8") if isinstance(x, bytes) else x)


In [67]:
# --- 7. Detect outliers via z-score ---
z_scores = np.abs((X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0))
threshold = 2.0  

mask = (z_scores < threshold).all(axis=1)
X_clean = X_scaled[mask]
y_clean = y[mask]

print(f"Removed {(~mask).sum()} outliers out of {len(mask)} ({100*(~mask).sum()/len(mask):.2f}%)")

Removed 187 outliers out of 569 (32.86%)


In [68]:
print(f"n_samples, n_features (after outlier removal): {X_clean.shape}")
np.save(f"{base_path}/arrays/wdbc.npy", X_clean)
D = squareform(pdist(X_clean, metric="euclidean"))

n_samples, n_features (after outlier removal): (382, 30)


In [69]:
save_results(path = f"{base_path}/results/wdbc.pkl", diss_matrix=D)

Number of cluster (K): 15: 100%|██████████| 14/14 [02:50<00:00, 12.18s/it]


In [70]:
results_stats(results=pd.read_pickle(f"{base_path}/results/wdbc.pkl"), n_classes=2, labels=y_clean)

K         min|C|    ARI       AMI       ASW       UB        WCRE      UBm       WCREm     
------------------------------------------------------------------------------------------
2         46        0.427     0.386     0.348     0.621     0.440     0.455     0.236     


# Customers
* Scaling: Robust Scaler
* Outlier removal: Yes
* n_samples: 378
* n_features: 7
* n_classes: 2
* Access to labels: yes
* Metric: Cosine

In [71]:
df = pd.read_csv(f"{base_path}/datasets/customers/data.csv")
y = df["Channel"]

In [72]:
df = df.drop(columns=["Channel"])
X = df.to_numpy()
X_clean = RobustScaler().fit_transform(X)

In [73]:
z_scores = np.abs(stats.zscore(X_clean))
mask = (z_scores < 2).all(axis=1)  
X_clean = X_clean[mask]
y_clean = y[mask]
print(f"Removed {(~mask).sum()} outliers out of {len(mask)} ({100*(~mask).sum()/len(mask):.2f}%)")

Removed 62 outliers out of 440 (14.09%)


In [74]:
print(f"n_samples, n_features (after outlier removal): {X_clean.shape}")
np.save(f"{base_path}/arrays/customers.npy", X_clean)
D = squareform(pdist(X_clean, metric="cosine"))

n_samples, n_features (after outlier removal): (378, 7)


In [75]:
save_results(path = f"{base_path}/results/customers.pkl", diss_matrix=D)

Number of cluster (K): 15: 100%|██████████| 14/14 [02:33<00:00, 10.97s/it]


In [76]:
results_stats(results=pd.read_pickle(f"{base_path}/results/customers.pkl"), n_classes=2, labels=y_clean)

K         min|C|    ARI       AMI       ASW       UB        WCRE      UBm       WCREm     
------------------------------------------------------------------------------------------
2         170       0.394     0.366     0.400     0.951     0.579     0.574     0.303     


# Dermatology
* Scaling: Standard Scaler
* Outlier removal: No
* n_samples: 366
* n_features: 34
* n_classes: 6
* Access to labels: yes
* Metric: Cosine

In [77]:
# --- 1. Load the ARFF file ---
data, meta = arff.loadarff(f"{base_path}/datasets/dermatology/dermatology.arff")

# --- 2. Convert to DataFrame ---
df = pd.DataFrame(data)

# --- 3. Decode byte strings (ARFF nominal data often comes as bytes) ---
df = df.applymap(lambda x: x.decode("utf-8") if isinstance(x, bytes) else x)

# --- 4. Replace missing values ('?') and convert to numeric ---
df = df.replace('?', pd.NA)
df = df.astype(float)

# --- 5. Separate features and labels ---
X = df.drop(columns=["class"])
y = df["class"]

# --- 6. Impute missing Age values ---
imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(X)

# --- 7. Standardize and then normalize for cosine distance ---
X_scaled = StandardScaler().fit_transform(X_imputed)
X_cosine = normalize(X_scaled, norm="l2")
X_cosine.shape

/var/folders/40/l50vx_fn2nd9hv4y_ssvkhn80000gn/T/ipykernel_59080/2128838603.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.decode("utf-8") if isinstance(x, bytes) else x)


(366, 34)

In [78]:
np.save(f"{base_path}/arrays/dermatology.npy", X_cosine)
D = squareform(pdist(X_cosine, metric="cosine"))

In [79]:
save_results(path = f"{base_path}/results/dermatology.pkl", diss_matrix=D)

Number of cluster (K): 15: 100%|██████████| 14/14 [02:02<00:00,  8.74s/it]


In [80]:
results_stats(results=pd.read_pickle(f"{base_path}/results/dermatology.pkl"), n_classes=6, labels=y)

K         min|C|    ARI       AMI       ASW       UB        WCRE      UBm       WCREm     
------------------------------------------------------------------------------------------
6         21        0.850     0.875     0.455     0.836     0.456     0.704     0.354     


# Optdigits
* Scaling: Standard Scaler
* Outlier removal: Yes
* n_samples: 2296
* n_features: 64
* n_classes: 10
* Access to labels: yes
* Metric: Euclidean

In [8]:
# Load training and test data
train_df = pd.read_csv(f"{base_path}/datasets/optdigits/optdigits.tra", header=None)
test_df = pd.read_csv(f"{base_path}/datasets/optdigits/optdigits.tes", header=None)

# Combine for clustering (drop labels if you want unsupervised learning)
df = pd.concat([train_df, test_df], ignore_index=True)

# Separate features and labels
X = df.iloc[:, :-1]  # 64 features
y = df.iloc[:, -1]   # labels 0-9

In [9]:
X = StandardScaler().fit_transform(X)  

In [10]:
mean = X.mean(axis=0)
std = X.std(axis=0)
std[std == 0] = 1  # avoid division by zero

z_scores = np.abs((X - mean) / std)
mask = (z_scores < 2.0).all(axis=1)
X = X[mask]
y = y[mask]
print(f"Removed {(~mask).sum()} outliers out of {len(mask)} ({100*(~mask).sum()/len(mask):.2f}%)")

Removed 3324 outliers out of 5620 (59.15%)


In [11]:
print(f"n_samples, n_features (after outlier removal): {X.shape}")
print(f"# unique labels: {len(np.unique(y))}")
np.save(f"{base_path}/arrays/optdigits.npy", X)
D = squareform(pdist(X, metric="euclidean"))

n_samples, n_features (after outlier removal): (2296, 64)
# unique labels: 10


In [12]:
save_results(path = f"{base_path}/results/optdigits.pkl", diss_matrix=D)

Path exists. Aborting.


In [13]:
results_stats(results=pd.read_pickle(f"{base_path}/results/optdigits.pkl"), n_classes=10, labels=y)

K         min|C|    ARI       AMI       ASW       UB        WCRE      UBm       WCREm     
------------------------------------------------------------------------------------------
10        50        0.803     0.819     0.180     0.620     0.709     0.463     0.611     


# Heart statlog
* Scaling: Standard Scaler
* Outlier removal: Yes
* n_samples: 231
* n_features: 13
* n_classes: 2
* Access to labels: yes
* Metric: Euclidean

In [81]:
data, meta = arff.loadarff(f"{base_path}/datasets/heartstatlog/heart-statlog.arff")

# --- 2. Convert to DataFrame ---
df = pd.DataFrame(data)

In [82]:
y = df["class"]
y = y.str.decode('utf-8')
# Map to integers
y = y.map({'absent': 0, 'present': 1})

In [83]:
# Drop the target
X = df.drop(columns=["class"])

# One-hot encode nominal columns
nominal_cols = ["chest", "resting_electrocardiographic_results", "slope", "thal"]
X = pd.get_dummies(X, columns=nominal_cols, drop_first=True)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [84]:
# --- 6. Detect outliers via z-score ---
z_scores = np.abs((X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0))
threshold = 3.0  

mask = (z_scores < threshold).all(axis=1)
X_clean = X_scaled[mask]
y_clean = y[mask]

print(f"Removed {(~mask).sum()} outliers out of {len(mask)} ({100*(~mask).sum()/len(mask):.2f}%)")

Removed 39 outliers out of 270 (14.44%)


In [85]:
print(f"n_samples, n_features (after outlier removal): {X_clean.shape}") # > 13 features due to one-hot encoding
np.save(f"{base_path}/arrays/heart_statlog.npy", X_clean)
D = squareform(pdist(X_clean, metric="euclidean"))
upper_bound(D)

n_samples, n_features (after outlier removal): (231, 18)


np.float64(0.6003930865824887)

In [86]:
save_results(path = f"{base_path}/results/heart_statlog.pkl", diss_matrix=D)

Number of cluster (K): 15: 100%|██████████| 14/14 [00:24<00:00,  1.77s/it]


In [87]:
results_stats(results=pd.read_pickle(f"{base_path}/results/heart_statlog.pkl"), n_classes=2, labels=y_clean)

K         min|C|    ARI       AMI       ASW       UB        WCRE      UBm       WCREm     
------------------------------------------------------------------------------------------
2         83        0.447     0.349     0.161     0.600     0.731     0.279     0.422     
